In [1]:
import pandas as pd
import numpy as np

t0 = pd.read_excel('C:/Users/Satvik Jha/Desktop/Python_test/Table_0.xlsx')
t2 = pd.read_excel('C:/Users/Satvik Jha/Desktop/Python_test/Table_2.xlsx')

table_ex = pd.concat([t0,t2],ignore_index = True)
table_ex.dropna(subset=['Acctg Date', 'Date'], inplace=True)
table_ex['Acctg Date'] = pd.to_datetime(table_ex['Acctg Date'],errors='coerce',format = '%Y-%m-%d')
table_ex['Date'] = pd.to_datetime(table_ex['Date'],errors='coerce',format = '%Y-%m-%d')
table_ex.head()

,Voucher,Seq,Type,Account,Func Unit,Amount,Currency,Acctg Date,Date,Line #,Origin
0,69649,0,4P4,111999,410851.0,588.60,EUR,2020-07-07,2020-07-07,55,ONL
1,19849,0,4P4,111999,411203.0,30250.00,EUR,2020-07-07,2020-07-07,56,ONL
2,69457,0,4P4,111999,411203.0,8848.53,EUR,2020-07-07,2020-07-07,56,ONL
3,69566,0,4P4,111999,411203.0,58550.58,EUR,2020-07-07,2020-07-07,56,ONL
4,69691,0,4P4,111999,411203.0,5580.95,EUR,2020-07-07,2020-07-07,57,ONL


In [2]:
table_csv = pd.read_csv('C:/Users/Satvik Jha/Desktop/Python_test/Table_1.csv')
table_csv['Acctg Date'] = pd.to_datetime(table_csv['Acctg Date'],errors='coerce',format = '%d/%m/%Y')
table_csv['Date'] = pd.to_datetime(table_csv['Date'],errors='coerce',format = '%d/%m/%Y')
table_csv.head()

,Voucher,Seq,Type,Account,Func Unit,Amount,Currency,Acctg Date,Date,Line #,Origin
0,14868,0,?45,197719,NaN,-3535.53,EUR,2019-06-01,2019-06-01,1,ONL
1,14869,0,?45,197719,NaN,-6265.99,EUR,2019-06-01,2019-06-01,1,ONL
2,14849,0,?45,197719,NaN,-5559.58,EUR,2019-06-01,2019-06-01,1,ONL
3,14856,0,?45,197719,NaN,4320.22,EUR,2019-06-01,2019-06-01,1,ONL
4,14888,0,?45,197719,NaN,5943.26,EUR,2019-06-01,2019-06-01,1,ONL


In [3]:
My_File = pd.concat([table_csv,table_ex], ignore_index = True)

My_File.drop_duplicates(keep = 'last')

My_File['Acctg Date'] = My_File['Acctg Date'].values.astype('datetime64[D]')

My_File['Date'] = My_File['Date'].values.astype('datetime64[D]')

My_File["Func Unit"].fillna(1337,inplace = True)

My_File['number_of_days'] = My_File['Acctg Date'] - My_File['Date']

My_File.sample(5)

,Voucher,Seq,Type,Account,Func Unit,Amount,Currency,Acctg Date,Date,Line #,Origin,number_of_days
8027,15986,0,D57,564999,220101.0,5282.96,EUR,2019-08-13,2019-08-13,3,ONL,0 days
14261,16894,0,D57,499699,220101.0,3968.88,EUR,2019-10-09,2019-10-09,7,ONL,0 days
29592,61681,0,V0P,146191,100112.0,0.00,GBP,2020-09-24,2020-09-24,4,ICO,0 days
24137,18978,0,RXL,596799,411203.0,55.44,PLN,2020-01-13,2020-01-13,11,ICO,0 days
26514,69958,0,D57,567999,410201.0,-3953.65,EUR,2020-08-01,2020-08-01,38,ONL,0 days


In [4]:
import pandas_market_calendars as mcal

My_File['Acctg Date'] = pd.to_datetime(My_File['Acctg Date'])
My_File['Date'] = pd.to_datetime(My_File['Date'])

uk_calendar = mcal.get_calendar('XLON')

uk_market_days = uk_calendar.valid_days(start_date=My_File['Acctg Date'].min(), 
                                         end_date=My_File['Date'].max())

uk_holidays = uk_calendar.holidays().holidays

My_File['Number_of_Busdays'] = np.busday_count(My_File['Acctg Date'].dt.date.values.astype('datetime64[D]'),
                                           My_File['Date'].dt.date.values.astype('datetime64[D]'),
                                           holidays=uk_holidays)

My_File.head()


,Voucher,Seq,Type,Account,Func Unit,Amount,Currency,Acctg Date,Date,Line #,Origin,number_of_days,Number_of_Busdays
0,14868,0,?45,197719,1337.0,-3535.53,EUR,2019-06-01,2019-06-01,1,ONL,0 days,0
1,14869,0,?45,197719,1337.0,-6265.99,EUR,2019-06-01,2019-06-01,1,ONL,0 days,0
2,14849,0,?45,197719,1337.0,-5559.58,EUR,2019-06-01,2019-06-01,1,ONL,0 days,0
3,14856,0,?45,197719,1337.0,4320.22,EUR,2019-06-01,2019-06-01,1,ONL,0 days,0
4,14888,0,?45,197719,1337.0,5943.26,EUR,2019-06-01,2019-06-01,1,ONL,0 days,0


In [5]:
FXrates = pd.read_csv('C:/Users/Satvik Jha/Desktop/Python_test/FXrates.csv')
revised = My_File.merge(FXrates, left_on = 'Currency', right_on = 'Currency',how = 'left')
revised = (revised.assign(PLN = revised['Amount'] * revised['Per USD'] ))
revised.sample(5)

,Voucher,Seq,Type,Account,Func Unit,Amount,Currency,Acctg Date,Date,Line #,Origin,number_of_days,Number_of_Busdays,Per USD,PLN
5224,15566,0,4P4,159919,220603.0,-232.63,PLN,2019-07-22,2019-07-22,8,ICO,0 days,0,3.6826,-856.683238
26280,69977,0,4P4,159919,411203.0,-4048.55,PLN,2020-08-01,2020-08-01,4,ICO,0 days,0,3.6826,-14909.190230
35999,18518,0,D57,499699,100811.0,35.40,PLN,2020-03-01,2020-03-01,254,ICO,0 days,0,3.6826,130.364040
40392,19155,0,Ź45,197719,1337.0,-4696.84,EUR,2020-04-14,2020-04-14,1,ONL,0 days,0,0.8378,-3935.012552
33436,18756,0,V1R,117997,410201.0,530.50,EUR,2020-02-04,2020-02-04,1,ONL,0 days,0,0.8378,444.452900


In [8]:
import os


folder_name = 'results'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    
    
for type_value in revised['Type'].unique():
    type_df = revised[revised['Type'] == type_value]
    
    type_value_cleaned = type_value.replace('?', '_')
    
    file_name = f'Table_{type_value_cleaned}.xlsx'
    
    
    file_path = os.path.join(folder_name, file_name)
    type_df.to_excel(file_path, index=False)

    print(f"File '{file_name}' saved successfully.")

File 'Table__45.xlsx' saved successfully.
File 'Table_V1R.xlsx' saved successfully.
File 'Table_4P4.xlsx' saved successfully.
File 'Table_1UR.xlsx' saved successfully.
File 'Table_1UP.xlsx' saved successfully.
File 'Table_D57.xlsx' saved successfully.
File 'Table_V1_R.xlsx' saved successfully.
File 'Table_V0P.xlsx' saved successfully.
File 'Table_V0_P.xlsx' saved successfully.
File 'Table_RXG.xlsx' saved successfully.
File 'Table_RXL.xlsx' saved successfully.
File 'Table_V1WR.xlsx' saved successfully.
File 'Table_R54.xlsx' saved successfully.
File 'Table_V0WP.xlsx' saved successfully.
File 'Table_Ź45.xlsx' saved successfully.
File 'Table_V1ŹR.xlsx' saved successfully.
File 'Table_V0ŹP.xlsx' saved successfully.
